In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import graph_tool.all as gt
import sys
# sys.insert_path('/Users/glover.co/Documents/laszlo/NetDesign/scripts/')
import assembly_tree as at
import mcmc

In [2]:
X = np.vstack([np.eye(3) for i in range(2)])
O = np.array([[0,1,1],[1,0,1],[1,1,2]])
capacity = O.sum(axis=1,dtype=int)
target = nx.Graph()
target.add_nodes_from(np.arange(6))
target.add_edges_from([[0,1],[1,2],[2,0],[3,4],[4,5],[5,3],[2,5]])

In [3]:
ex0 = mcmc.AssemblyTree(target,X,O,capacity)

In [4]:
ex0.update_tree()

In [5]:
ex0.Tree.leaves()

[Node(tag=1, identifier=1, data=<mcmc.AssemblyNode object at 0x309209bd0>),
 Node(tag=2, identifier=2, data=<mcmc.AssemblyNode object at 0x309209e50>),
 Node(tag=3, identifier=3, data=<mcmc.AssemblyNode object at 0x1045d0c30>),
 Node(tag=4, identifier=4, data=<mcmc.AssemblyNode object at 0x3092782b0>)]

In [17]:
ex0.Tree.show()

0
├── 1
├── 2
├── 3
└── 4



In [16]:
ex0.Tree.get_node(4).data.logP

[np.float64(0.0)]

In [8]:
sub_nodes = ex0.Tree.get_node(1).data.nodes
ex0.target_A[sub_nodes,:][:,sub_nodes]

array([[0, 0],
       [0, 0]])

In [9]:
x = 1
y = 2
z = 3
true_list = [x,y,z]
for j, s in enumerate(true_list):
    true_list[j] = 4

In [10]:
true_list

[4, 4, 4]

In [11]:
at.draw_network(ex0.Tree.get_node(0).data.subgraph[1],ex0.X)

IndexError: list index out of range

In [ ]:
ex0.Tree.